# Chapter 3: Djagios – a Nagios Clone in Django

In this chapter, we'll be creating a server status monitoring solution similar to Nagios. It is understandable if you have never heard of Nagios as it is not something that comes up in day-to-day conversations between web developers. In a nutshell, Nagios can tell you the status of your servers (which can be in the thousands) in one screen. You can configure alerts based on conditions, for example, if some critical
server becomes unresponsive, so that you can be on top of problems before your users start to notice any service degradation. Nagios is an amazing piece of software that is used by millions of organizations around the world.

Our goal in this chapter is to create something similar, albeit very simple. Our Nagios clone, creatively named Djagios, will allow users to set up monitoring for simple stats from their servers. We will allow the monitoring of the following:

- System load
- Disk usage

We will also be developing a web page where this data will be shown to the user in a nice tabular format. The user will also see an overview of how their servers are doing, and if any alerts are active on those systems.

Here are some of the things that we will be looking at in this chapter:

- What Django management commands are and how to create a custom one
- Using the Django shell to test out small pieces of code quickly
- Complicated validation of Django Model fields
- Slightly more complicated uses of the built-in generic views
- Creating an API endpoint to accept data from external sources
- Securing these API endpoints with simple shared secret keys
- Testing API endpoints using simple tools

## Code pack

This chapter's code pack has a basic Django application setup with a SQLite database already configured. However, there isn't much code in the code pack because this chapter did not require user accounts or any other pre-existing setup. You can unzip the code pack, create a new virtual environment, activate it, and run the following commands from within the code folder to get up and running:



```sh
> pip install django
> python manage.py migrate
```

## Requirements

Let's talk a bit about what we want from our end product before we start writing
up some code. As mentioned before, we're looking to create a server monitoring
solution. What exactly will it do? How can we implement the required functionality?

As our inspiration for Djagios is Nagios, let's look at how Nagios works. While
Nagios is a huge application with understandably complex programming, it is in the
end a client-server application. The server, which is just another computer, contains
the Nagios installation. The clients, which are the systems that you want to monitor,
run small plugin scripts that gather data and push it to the server. The server takes
these data points and, based on how it's configured, sends out alerts if required. It
also stores these data points and can show them in a simple tabular layout, giving
you an instant overview of all the computer systems in your infrastructure.



We'll be creating something similar. Our server will be a Django application that will
accept data points using an HTTP endpoint. The application will also include a web
page where all these data points will be shown next to the client they came from. Our
clients will be simple shell scripts that upload the data to our server.

> For the rest of the chapter, I will call the Django application,
the server, and the systems that you want to monitor, the
node. These are common terms used in many other projects
that you will encounter in your programming career and they
often mean similar things in those other projects.

Instead of developing all of these things at once, we'll take an incremental approach.
We'll first create the models to store our data points. Next, instead of moving directly
to creating the HTTP endpoint to accept the data points and client-side plugin
scripts, we'll take a simpler approach and come up with a way to generate some
fake data to test. An finally, we will create the web page to show our users the latest
status of the client nodes and the triggered alerts.



With fake data to test, we can be confident that our status page and alerting system
are working correctly. We can then move forward to the next step, which will be
creating the HTTP endpoint to gather data points from clients and client-side plugin
scripts.



In real-world projects, this incremental approach to building software systems
is often the best way to complete projects. Create simple features and test them
extensively to make sure that they work. Once you are confident of their correctness,
add more functionality and repeat the testing phase. This way is analogous to how
a tall building is constructed. If you are confident that the foundation is strong, you
can build one floor at a time without worrying if the whole thing will fall down on
your head.

## The model

What information should we record for the data points? We definitely need the name
of the node that is sending the data. We also need to record the time at which we
got the data point so that we can figure out the latest status of the node. Of course,
we need to know the type and value of the data point. The type of the data point is
simply the name of the quantity that we are measuring, such as CPU usage, memory
usage, uptime, and so on.


For now, I think these are all the things that we need to measure:
- Node name
- Date and time
- Type
- Value

While thinking about which fields we needed in our model, I thought of another
approach. It involved having a different model for each data point type, so we could
have Django models named SystemLoad, MemoryUsage, DiskUsage, Uptime, and so
on. However, once I thought about it further, I figured out that doing so would be
very restrictive as now we'd need to define a new model every time we wanted to
measure something new. Having the type of the data point as another field in our
model gives us a lot of flexibility in terms of recording new types of information.


As you'll see later on, there are pros and cons to both approaches.

Let's start a new Django application in our project. In your command line, type the
following, making sure that your virtual environment is activated and that you are in
the project folder:



```
> python manage.py startapp data_collector
```

Add this new app to the list of INSTALLED_APPS in djagios/settings.py, and then
add our data point model to data_collector/models.py:



```
class DataPoint(models.Model):
    node_name = models.CharField(max_length=250)
    datetime = models.DateTimeField(auto_now_add=True)
    data_type = models.CharField(max_length=100)
    data_value = models.FloatField()
```



Save the file and then run migrations to add it to our database:



```
> python manage.py makemigrations data_collector
> python manage.py migrate
```



While the model is pretty simple, there is one thing you should be aware of. To
keep things simple, I made the decision to store only numeric values; hence, the
data_value field is of the FloatField type. If this were a real-world project, you
would have a set of requirements that would dictate if you could make the same
compromise. You might have to record text values as well, for instance, the status
of some service you might be running. For Djagios, we are OK with numeric values
only as all the stats that we want to measure are just numbers.

## Fake data generation



Before moving on to the next step, which is creating the status page, we should come
up with a way to generate some fake data. This will help us while creating the status
page and debug any issues that we encounter along the way. Without any data, we
could spend time creating the perfect status page, only to find out later when we add
data that some aspect of it, such as the design or data layout scheme, won't work.

## Django management commands



Django has a very useful feature called management commands. It allows us to
create Python scripts that can interact with the rest of the Django app code we write,
including our models.


The reason we can't just write a simple Python script that imports our models is that
Django, like all web frameworks, has many dependencies and requires a complex
setup to make sure that every thing is configured properly before we can use its
features. For example, accessing the database depends on Django knowing where
the settings file is because it contains the configuration for the database. Without
knowing how to read the database, you can't query the models.

Let's conduct a little test. Make sure that your virtual environment is activated
and that you are in the project directory. Next, start up the Python shell by typing
the following:

```sh
> python
```

This will start a new Python interactive shell where you can type Python statements
and have them executed immediately. You'll know that you are in a Python shell
because your prompt will change to >>>. Now, let's try importing our DataPoint
model in this shell:

```sh
>>> from data_collector.models import DataPoint
```

Press Enter and you might be surprised by the huge error message that is printed.
Don't worry, you don't need to read all of it. The most important part is the last
line. It will be similar to this (though not exactly as it might change slightly between
different Django versions):

> django.core.exceptions.ImproperlyConfigured: Requested setting
DEFAULT_INDEX_TABLESPACE, but settings are not configured. You must
either define the environment variable DJANGO_SETTINGS_MODULE or call
settings.configure() before accessing settings.

It looks intimidating, but let's look at it in parts. The first part before the colon : is
the exception name. Here, Django raised the ImproperlyConfigured exception.
Then there is a sentence telling you that some setting was requested, but settings are
not yet configured. The last sentence is a helpful message about how to fix this.



While you can take the steps that are listed in the error message and get your scripts
to run outside of Django, using a management command is almost always the best
option. With a management command, you don't need to set up Django manually.
It's done for you automatically before your script is run and you can avoid having
to worry about setting environment variables such as DJANGO_SETTINGS_MODULE or
calling settings.configure().



To create a new management command, we first have to create two new Python
modules to hold our command script. Type the following in your command line
from the project root directory:



```sh
> mkdir -p data_collector/management/commands
> touch data_collector/management/__init__.py
> touch data_collector/management/commands/__init__.py
```

What these commands do is create a module called management under the
data_collector module folder, and then create another module called commands
in the management module. We do this by first creating the folders for the modules
and then creating empty `__init__.py` files so that Python recognizes these folders
as modules.


Next, let's try creating a simple management command that just prints out a list ofthe data points that we have in our database so far. Create a new `data_collector/management/commands/sample_data.py` file and give it the following content:

```sh
from django.core.management import BaseCommand
from data_collector.models import DataPoint

class Command(BaseCommand):
    def handle(self, *args, **options):
        print('All data points:')
        print(DataPoint.objects.all())
```

Save this file and go back to your command prompt. Then run the following
command:



```sh
> python manage.py sample_data
```

You should see the following output:   
```
All data points:    
[]

```

That's it. That's all there is to creating Django management commands. As you can
see, we were able to use methods on our DataPoint model in a script that was run
on the command line, as opposed to being run as part of an HTTP response view. A
few things to note about Django management commands are as follows:


- Your command has the same name as the name of the file that contains the command.
- In order to be a valid management command, the source file should always define a Command class, which will be a base class of BaseCommand.
- Django will call the handle method of your Command class. This method is the starting point of whatever functionality you want to provide from your script.

We'll take a look at the `*args and **options` parameters next when we modify the sample_data.py command to actually add sample data. If you want further information about Django management commands, you should look at the documentation at https://docs.djangoproject.com/en/stable/howto/custommanagement-commands/.



Let's modify our command class to add fake data. Here's the modified Command
class code:

```sh
class Command(BaseCommand):
    def add_arguments(self, parser):
        parser.add_argument('node_name', type=str)
        parser.add_argument('data_type', type=str)
        parser.add_argument('data_value', type=float)
    def handle(self, *args, **options):
        node_name = options['node_name']
        data_type = options['data_type']
        data_value = options['data_value']
        new_data_point = DataPoint(node_name=node_name, data_type=data_type, data_value=data_value)
        new_data_point.save()
        print('All data points:')
        print(DataPoint.objects.all())
```

There are a couple of new things here. Let's look at the add_arguments method first.
Most management commands need arguments to do something useful. As we're
adding sample data to the database, our command will need the values to add. These
are provided to the command in the form of arguments on the command line. If you
don't have much experience with working with command lines, arguments are all
the things that you type after the command name. For example, let's take a look at
the command we use to create new Django applications in a project:



```sh
> python manage.py startapp APP_NAME
```


Here, we use the startapp Django management command, which is a built-in
command, and the app name is an argument to the app.


We want our custom command to accept three arguments: the node name, data type,
and value of the data point. In the add_arguments method, we tell Django to require
and parse three arguments for this command.


The options parameter to the handle method is a dictionary that holds all the
arguments that were defined and passed to the command by the user. In the handle
method, we simply assign the values for each option to a variable. In case the user
misses or adds extra arguments while calling the command, Django will print an
error message as well as inform them of what the required parameters are. For
example, if I call the command now without any parameter, here's what happens:

```sh
> python manage.py sample_data
usage: manage.py sample_data [-h] [--version] [-v {0,1,2,3}]
                    [--settings SETTINGS] [--pythonpath
PYTHONPATH]
                    [--traceback] [--no-color]
                    node_name data_type data_value
manage.py sample_data: error: the following arguments are required:
node_name, data_type, data_value
```

This is useful information if the user forgets how to use a management command.


Now that we have the argument values in variables, we create a new data point and
save it. Finally, we print out all the data points that we have in the database. Let's try
running our command now and see what output we get:

```sh
> python manage.py sample_data web01 load 5
All data points:
[<DataPoint: DataPoint object>]
```

While the command successfully created a new data point, the output isn't very
helpful. We have no idea what information this data point holds. Let's fix this.

## A better model representation


Whenever Django prints out a model instance, it first tries to see if there is a `__str__`
method defined for the model class. If it finds this method, it uses the output from it;
otherwise, it falls back to a default implementation that just prints the class name, as
we can see here. To make Django print out a more useful representation for our data
point model, add this `__str__` method to our `DataPoint` model class:

```sh
def __str__(self):
    return 'DataPoint for {}. {} = {}'.format(self.node_name, self.data_type, self.data_value)
```

Let's run our sample_data command again now and see how its output has changed:

```sh
> python manage.py sample_data web01 load 1.0
All data points:
[<DataPoint: DataPoint for web01. load = 5.0>, <DataPoint: DataPoint
for web01. load = 1.0]
```

That's better. Now we see that the data point we added has been saved to the
database correctly. Go ahead and create a couple of more data points. Use as many
different node names as you want, but try to keep the data type limited to one of
load or disk_usage as we'll be creating code specific to these data types later. Here's
the sample data that I added to my database for reference:

|Node name| Data type| Data value|
|:---:|:---:|:---|
|web01| load| 5.0|
|web01| load| 1.0|
|web01| load| 1.5|
|web01| disk|`_usage 0.5`|
|web02| load| 7.0|
|web02| load| 9.0|
|web02| disk_usage| 0.85|
|dbmaster| disk_usage| 0.8|
|dbmaster| disk_usage| 0.95|

Now that we have a way to add sample data and added some to our database, let's
create the status page that will show all this data to our users.

## Status page


Our status page needs to show the user the status of their complete infrastructure in
one view. To do so, a table feels like the appropriate design component. As the most
important information to the user is going to be the latest status of their servers, our
status page will need to show only one table row per node and list only the latest
values for each different data type that we have in the database for that node.


For the sample data that I added to my database, we would ideally want a table
similar to this on the status page:

![ ](img/VPLNWDR.png)

As you can see, we only mention each node once, and group the different data types
so that all information about a node is shown in one place and the user doesn't have
to search through the table to find what they are looking for. As a bonus, we also
show the last updated time in a nice way instead of just showing the time of the
latest data point.


If you are thinking that showing our data points in a nice and consolidated manner
like this will not be simple, I'm afraid you are right. We could get all the data points
from the database in one query using `DataPoint.objects.all()` and then group
them in our Python code, but that would be inefficient once the number of data
points in our database grows. For server monitoring solutions, it isn't uncommon
to have a couple of million data points. We can't go about getting and grouping all
the million data points every time the user wants to see the status page. That would
make loading the page unbearably slow.


Luckily for us, SQL—the language used to query data from databases—provides us
with some very powerful constructs that we can use to get just the information we
want, without having to go through all the rows of data that we could have in our
data points table. Let's think about what we need.

For starters, we'd like to know the different node names in our database. For each
node name, we also need to know the types of data that are available for it. In our
example, while both **web01** and **web02** have the **load** and **disk_usage** data type
available, the **dbmaster** node only has data for the **disk_usage** data type (or metric).
For cases like this, the SQL language provides us with a DISTINCT query clause.
Adding DISTINCT to our query instructs the database to return unique rows only.
That is, all duplicate rows are only returned once. This way, we can get a list of all the
different nodes and data types in our database without having to go over each record.


We need to experiment a bit to figure out how to translate the SQL query into
something we can use with the Django ORM. We could write our view code and
then keep changing it to figure out the right way to get the data we want, but that is
very cumbersome. Instead, Django provides us with a very convenient shell to do
these kinds of experimentations.


If you remember, at the start of this chapter, I showed you why you couldn't just
start a Python shell and import the model. Django complained about not having been
set up properly before being used. Instead, Django has its own way of launching a
Python shell, one that makes sure that all the dependencies of setting up Django are
met before you start using the shell. To start this shell, type the following:

```sh
> python manage.py shell
```

Like before, this will put you in a Python shell, which you can tell by the changed
prompt. Now, let's try importing our DataPoint model:


```sh
>>> from data_collector.models import DataPoint
```


This time you shouldn't get any errors. Now type the following:


```
>>> DataPoint.objects.all()
[<DataPoint: DataPoint for web01. load = 5.0>, <DataPoint: DataPoint
for web01. load = 1.0>, <DataPoint: DataPoint for web01. load = 1.5>,
<DataPoint: DataPoint for web02. load = 7.0>, <DataPoint: DataPoint for
web02. load = 9.0>, <DataPoint: DataPoint for dbmaster. disk_usage =
0.8>, <DataPoint: DataPoint for dbmaster. disk_usage = 0.95>, <DataPoint:
DataPoint for web01. disk_usage = 0.5>, <DataPoint: DataPoint for web02.
disk_usage = 0.85>]
```


As you can see, you can query the model and see the output of the query
immediately. The Django shell is one of the most useful components in Django, and
you will often find yourself experimenting in the shell to figure out the correct way
to do something before you write the final code in your views.

So, back to our problem of getting distinct node names and data types from our
database. If you search the Django documentation for the distinct keyword, you
should see this link in the results:
```
https://docs.djangoproject.com/en/stable/ref/models/querysets/#distinct.
```



If you read what the documentation says, you should figure out that this is exactly
what we need in order to use the DISTINCT clause. But how do we use it? Let's try it
out in the shell:



```sh
>>> DataPoint.objects.all().distinct()
[<DataPoint: DataPoint for web01. load = 5.0>, <DataPoint: DataPoint
for web01. load = 1.0>, <DataPoint: DataPoint for web01. load = 1.5>,
<DataPoint: DataPoint for web02. load = 7.0>, <DataPoint: DataPoint for
web02. load = 9.0>, <DataPoint: DataPoint for dbmaster. disk_usage =
0.8>, <DataPoint: DataPoint for dbmaster. disk_usage = 0.95>, <DataPoint:
DataPoint for web01. disk_usage = 0.5>, <DataPoint: DataPoint for web02.
disk_usage = 0.85>]
```

Hmm? That didn't change anything. Why not? Let's think about what is happening
here. We asked Django to query the database for all the data points and then return
only one row for each duplicated data. If you are familiar with SQL, the distinct
clause works by comparing each field in the rows of data you selected. However, as
by default Django selects all the rows from a database table when querying a model,
the data that the SQL query sees also includes the primary key, which is by definition
unique for each row. This is why we see all of our data, even though we have used
the distinct clause.


In order to make use of the distinct clause, we need to limit the fields in the data that
we are asking the database to return to us. For our particular use case, we only need
to know the unique pairs of node name and data type. The Django ORM provides
another method, values, that we can use to limit the fields that Django selects. Let's
try it out first without the distinct clause to see what data it returns:


```sh
>>> DataPoint.objects.all().values('node_name', 'data_type')
[{'data_type': u'load', 'node_name': u'web01'}, {'data_type': u'load',
'node_name': u'web01'}, {'data_type': u'load', 'node_name': u'web01'},
{'data_type': u'load', 'node_name': u'web02'}, {'data_type': u'load',
'node_name': u'web02'}, {'data_type': u'disk_usage', 'node_name':
u'dbmaster'}, {'data_type': u'disk_usage', 'node_name': u'dbmaster'},
{'data_type': u'disk_usage', 'node_name': u'web01'}, {'data_type':
u'disk_usage', 'node_name': u'web02'}]
```

That seems to do the trick. Now our data only includes the two fields that we wanted
to run the distinct query on. Let's add the distinct clause as well and see what we get:


```sh
>>> DataPoint.objects.all().values('node_name', 'data_type').distinct()
[{'data_type': u'load', 'node_name': u'web01'}, {'data_type': u'load',
'node_name': u'web02'}, {'data_type': u'disk_usage', 'node_name':
u'dbmaster'}, {'data_type': u'disk_usage', 'node_name': u'web01'},
{'data_type': u'disk_usage', 'node_name': u'web02'}]
```


Voila! That seems to have done the trick. Now our Django ORM query only returns
unique pairs of node names and data types, which is exactly what we needed.


One important thing to note is that after we added the `values` method to the ORM
query, the returned data was no longer our `DataPoint` model class. Instead, it was
dictionaries with just the field values that we asked for. Thus, any functions that you
have defined on the model are not accessible on these dictionaries. If you think about
it, this is obvious because without the complete fields, Django has no way to populate
the model objects. It also won't matter if you listed all of your model fields in the
`values` method arguments. It will still only return dictionaries, not the model objects.


Now that we have figured out how to get the data in the format we want, without
having to loop over each row of data in our database, let's create the template, view,
and URL configuration for our status page. Starting with the view code, change
`data_collector/views.py` to have these contents:

```sh
from django.views.generic import TemplateView
from data_collector.models import DataPoint

class StatusView(TemplateView):
    template_name = 'status.html'
    def get_context_data(self, **kwargs):
        ctx = super(StatusView, self).get_context_data(**kwargs)
        nodes_and_data_types = DataPoint.objects.all().values('node_name', 'data_type').distinct()
        status_data_dict = dict()
        for node_and_data_type_pair in nodes_and_data_types:
            node_name = node_and_data_type_pair['node_name']
            data_type = node_and_data_type_pair['data_type']
            data_point_map = status_data_dict.setdefault(node_name,dict())
            data_point_map[data_type] = DataPoint.objects.filter(node_name=node_name, data_type=data_type).latest('datetime')
        ctx['status_data_dict'] = status_data_dict
        return ctx
```

It's a little complicated, so let's break it up into parts. First, we get a list of node name
and data type pairs, using the query we came up with before. The result of the query,
which we store in nodes_and_data_types, is similar to the following:


```sh
[{'data_type': u'load', 'node_name': u'web01'}, {'data_type': u'load',
'node_name': u'web02'}, {'data_type': u'disk_usage', 'node_name':
u'dbmaster'}, {
'data_type': u'disk_usage', 'node_name': u'web01'}, {'data_type':
u'disk_usage', 'node_name': u'web02'}]
```

As we've seen before, this is a list of all unique node name and data type pairs in our
database. So, as our dbmaster node doesn't have any data for the load data type, you
won't find that pair in this list. I'll explain in a while why running the distinct query
helps us reduce the amount of load we need to put on our database.


Next, we loop over each of these pairs; that's the for loop you can see in the code. For
each node name and data type pair, we run a query to get us the latest data point.
First, we filter down to only the data points that we are interested in—those that
match the node name and data type we specified. Then, we call the `latest` method
and get the most recently updated data point.


The latest method takes the name of a field, orders the query using this field, and
then returns the last row of data as per that ordering. It should be noted that `latest`
can work with any field type that can be ordered, including numbers, not just date
time fields.


I would like to point out the use of `setdefault` here. Calling `setdefault` on
a dictionary makes sure that if the key provided doesn't exist in the dictionary
already, the value passed as the second parameter is set for that key. This is a pretty
useful pattern that I and a lot of Python programmers use when we are creating a
dictionary in which all the keys need to have values of the same type—a dictionary
in this case.


This allows us to ignore the scenario in which the key does not previously exist in the
dictionary. Without using setdefault, we would first have to check whether the key
exists. If it did, we would modify that. If it didn't, we would create a new dictionary,
modify that, and then assign it to `status_data_dict.`

The `setdefault` method returns the value of the given key as well, whether it had
to set it to the default value or not. We save that in the `data_point_map` variable in
our code.


Finally, we add the `status_data_dict` dictionary to the context and return it.
We'll see in our template how we go over this data and display it to the user. I said
earlier that I would explain how the distinct query helped us reduce the load on
the database. Let's look at an example scenario. Assume that we have the same
three nodes in our infrastructure that we saw in our sample data: web01, web02,
and dbmaster. Let's say that we have had monitoring running for one whole day,
collecting stats for both load and disk usage on all three nodes every minute. Doing
the math, we should have the following:


Number of nodes x number of data types x number of hours x 60:


```
3 x 2 x 24 x 60 = 8640
```

Thus, our data base has 8,640 data point objects. Now, with the code that we have
in our view, we will only need to retrieve six data point objects from the database to
show the user an updated status page plus the one distinct query. If we had to get
all the data points, we would have to transfer the data for all those 8,640 data points
from the database, and then use only six of them.


For the template, create a folder called` templates` in the `data_collector` directory.
Then, create a `status.html` file in the template folder and give it the following
content:

```sh
{% extends "base.html" %}
{% load humanize %}
{% block content %}
<h1>Status</h1>
<table>
<tbody>
    <tr>
        <th>Node Name</th>
        <th>Metric</th>
        <th>Value</th>
        <th>Last Updated</th>
    </tr>
    {% for node_name, data_type_to_data_point_map in status_data_dict.items %}
        {% for data_type, data_point in data_type_to_data_point_map.items %}
            <tr>
                <td>{% if forloop.first %}{{ node_name }}{% endif %}</td>
                <td>{{ data_type }}</td>
                <td>{{ data_point.data_value }}</td>
                <td>{{ data_point.datetime|naturaltime }}</td>
            </tr>
        {% endfor %}
    {% endfor %}
</tbody>
</table>
{% endblock %}
```

There shouldn't be many surprises here. Ignoring the `load humanize` line, our
template simply creates a table using the data dictionary that we generated in our
view earlier. The two nested `for` loops might look a bit complicated, but looking at
the data that we are looping over should make things clear:


```sh
{u'dbmaster': {u'disk_usage': <DataPoint: DataPoint for dbmaster.disk_usage = 0.95>},
u'web01': {u'disk_usage': <DataPoint: DataPoint for web01. disk_usage= 0.5>,
u'load': <DataPoint: DataPoint for web01. load = 1.5>},
u'web02': {u'disk_usage': <DataPoint: DataPoint for web02. disk_usage= 0.85>,
u'load': <DataPoint: DataPoint for web02. load = 9.0>}}
```

The first for loop gets the node name and dictionary mapping data type to the latest
data point. The inner for loop then iterates over the data type and the latest data
point for the type and generates the table rows. We use the `forloop.first` flag
to print the node name only if the inner loop is running for the first time. Django
provides a few other useful flags related to for loops in templates. Take a look
at the documentation at` https://docs.djangoproject.com/en/stable/ref/templates/builtins/#for.`

When we print the `datetime` field for a data point, we use the `naturaltime` filter.
This filter is part of the humanize template tags provided as part of Django, which
is why we needed to use the `load humanize` line at the start of the template. The
`naturaltime` template filter outputs a date time value in a format that is easy for
humans to understand, for example, two seconds ago, one hour ago, 20 minutes ago,
and so on. You need to add `django.contrib.humanize to the list of `INSTALLED_
APPS` in `djagios/settings.py` before you can load the `humanize` template tags.

The final step in completing our status page is to add it to the URL configuration. As
the status page is what the user wants to see most often from a monitoring system,
let's make it the home page. Make the URL configuration file at `djagios/urls.py`
contain the following content:


```sh
from django.conf.urls import url
from data_collector.views import StatusView

urlpatterns = [
    url(r'^$', StatusView.as_view(), name='status'),
]
```


That's it. Run the development server:


```sh
> python manage.py runserver
```


Access the status page at http://127.0.0.1:8000. If you have followed the steps so
far, you should see a status page similar to the following one. Of course, your page
will show data from your database:

![status](img/status.png)

## Alerts



Now that we have a basic status page up and running, let's talk about allowing the
user to configure some alert conditions. For now, we will inform the user of any alert
condition by showing that node's information in red color on the status page.



First, we need to figure out what kind of alerts we want our users to set. From there,
we can figure out the technical details. So, let's think about it. Given that the data
types we are recording all have numeric values, it makes sense that the user should
be able to set thresholds. They can, for instance, set an alert if the system load of any
node goes above 1.0 or if the disk usage of a node goes above 80%.



Furthermore, maybe our users don't want to have the same alert conditions for
each node. A database node is expected to handle a lot of system load, so maybe our
users want to have a separate alert condition for the database node. Finally, if they
are doing maintenance on some of the nodes, they may want to stop some alerts
from triggering.



From all this, it seems that our alerts need to have fields to hold the following:

- Data type on which to trigger 
- Maximum value to trigger on
- Minimum value to trigger on
- Node name to trigger on
- If the alert is currently active

Of these, the data type and active status are required fields that should never be null.
The node name can be an empty string in which the alert conditions will be checked
for each node. If the node name is not an empty string, it will be checked for nodes
whose names match the provided string exactly.


As for the maximum and minimum values, one of those is required. This is so that
the user may set alerts for just the maximum value without having to care about the
minimum value for the data points. This will require manual validation in the model.

## The model

Let's look at the model. To keep things simple, we'll use the `data_collector` app
instead of creating a new one for alerts. Here is the code for our `Alert` model. Put
this in `data_collector/models.p`y after the `DataPoint` model code:


```sh
class Alert(models.Model):
    data_type = models.CharField(max_length=100)
    min_value = models.FloatField(null=True, blank=True)
    max_value = models.FloatField(null=True, blank=True)
    node_name = models.CharField(max_length=250, blank=True)
    is_active = models.BooleanField(default=True)
    def save(self, *args, **kwargs):
        if self.min_value is None and self.max_value is None:
            raise models.exceptions.ValidationError('Both min and max value can not be empty for an alert')
        super(Alert, self).save(*args, **kwargs)
```

Due to our special requirements for the min and max fields, we had to override the
`save` method. As you can probably tell, our custom `save` method raises an error if
both the min and max values are not set. As there is no way to express this condition
with normal Django field configuration, we have to override the `save` method and
add our custom logic here. This is a pretty common thing to do in Django if you have
some custom validation requirements that depend on more than one field.



One more thing to note is the `blank=True` parameter to the min and max
FloatField. This is required so that any model forms constructed from this model,
which we will use for the `create` and `update` views later, allow blank values for
these fields.


Make and run migrations to add this to your database.


```sh
> python manage.py makemigrations data_collector
> python manage.py migrate data_collector
```

## Management views



The users will need some views that they can use to manage alerts. They will need
pages to view all the alerts defined in the system, a page to create new alerts and edit
existing ones, and some way to delete alerts they no longer require. All of these are
achievable using the generic view provided by Django and some templates. Let's get
to it!



First, let's look at the list view first. Add this to `data_collector/views.py`:

```sh
class AlertListView(ListView):
    template_name = 'alerts_list.html'
    model = Alert
```

Remember to import `ListView` from `django.views.generic` and `Alert` from
`data_collector.models`. Next, create the `alerts_list.html` template file in
`data_collector/templates` and give it the following content:

```html
{% extends "base.html" %}

{% block content %}
<h1>Defined Alerts</h1>

{% if object_list %}
<table>
    <tr>
        <th>Data Type</th>
        <th>Min Value</th>
        <th>Max Value</th>
        <th>Node Name</th>
        <th>Is Active</th>
    </tr>
    
    {% for alert in object_list %}
    <tr>
        <td>{{ alert.data_type }}</td>
        <td>{{ alert.min_value }}</td>
        <td>{{ alert.max_value }}</td>
        <td>{{ alert.node_name }}</td>
        <td>{{ alert.is_active }}</td>
    </tr>
    {% endfor %}
</table>
{% else %}
<i>No alerts defined</i>
{% endif %}
{% endblock %}
```

Finally, edit `djagios/urls.py`. Import the new view, and then add this to the
URL patterns:



```sh
url(r'^alerts/$', AlertListView.as_view(), name='alerts-list'),
```

To test it out, open `http://127.0.0.1:8000/alerts/`. You should see the No
Alerts Defined message. The list view is pretty basic. The `ListVew` generic view renders a template with all the objects for the specified model, providing the list of objects in the `object_list` template context variable. Next, let's look at the view to create new alerts.

In the `data_collector/view.py` file, first import the following:


```sh
from django.core.urlresolvers import reverse
from django.views.generic import CreateView
```


Then add this view class:


```sh
class NewAlertView(CreateView):
    template_name = 'create_or_update_alert.html'
    model = Alert
    fields = [
        'data_type', 'min_value', 'max_value', 'node_name', 'is_active'
    ]
    def get_success_url(self):
        return reverse('alerts-list')
```

Nothing new in the view code. The template code is quite simple as well. Put this
code in `data_collector/templates/create_or_update_alert.html`:

```sh
{% extends "base.html" %}

{% block content %}
{% if object %}
<h1>Update Alert</h1>
{% else %}
<h1>New Alert</h1>
{% endif %}

<form action="" method="post">{% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="{% if object %}Update{% else %}
Create{% endif %}" />
    <a href="{% url 'alerts-list' %}">Cancel</a>
</form>
{% endblock %}
```

As in previous chapters, we use the `object` context variable to decide if this template
was used from `CreateView` or `UpdateView` and change some elements based on this.
Otherwise, it's pretty straightforward. Let's see the code for `UpdateView` as well:

```sh
class EditAlertView(UpdateView):
    template_name = 'create_or_update_alert.html'
    model = Alert
    fields = [
        'data_type', 'min_value', 'max_value', 'node_name', 'is_active'
    ]
    
    def get_success_url(self):
        return reverse('alerts-list')
```

It's an almost identical copy of the previous create view. Make sure that you have
imported the `UpdateView` generic view. We still need to add both of these views to
our URL configuration. In the `djagios/urls.py` file, import both `NewAlertView` and
`EditAlertView` and add these patterns:

```sh
url(r'^alerts/new/$', NewAlertView.as_view(), name='alerts-new'),
url(r'^alerts/(?P<pk>\d+)/edit/$', EditAlertView.as_view(),name='alerts-edit'),
```

Before we can test these views, we should add links to allow users to get to these
views. Modify the `alerts_list.html` template to match this code:

```html
{% extends "base.html" %}

{% block content %}
<h1>Defined Alerts</h1>

{% if object_list %}
<table>
    <tr>
        <th>Data Type</th>
        <th>Min Value</th>
        <th>Max Value</th>
        <th>Node Name</th>
        <th>Is Active</th>
    </tr>
    
    {% for alert in object_list %}
    <tr>
        <td>{{ alert.data_type }}</td>
        <td>{{ alert.min_value }}</td>
        <td>{{ alert.max_value }}</td>
        <td>{{ alert.node_name }}</td>
        <td>{{ alert.is_active }}</td>
        <td><a href="{% url 'alerts-edit' pk=alert.pk %}">Edit</a></td>
    </tr>
    {% endfor %}
</table>

{% else %}
<i>No alerts defined</i>
{% endif %}
<p><a href="{% url 'alerts-new' %}">Add New Alert</a></p>
{% endblock %}
```

Two new lines, which are highlighted, have been added. Now, let's see what our
alerts list page looks like. As before, open `http://127.0.0.1:8000/alerts/` in
your browser. You should see a page as follows:

![Difined Alerts](img/Difined Alerts.png)

Click on the Add New Alert link and you should see the form to create an alert. Fill
it in with some sample data and click on the Create button. If your form didn't have
any errors, you should be back to the alerts list view and your screen should now list
the new alert, as shown in the following screenshot:

![Difined Alerts new](img/Difined Alerts new.png)

All that's left now is to allow users the option to delete their alerts. For this, create
a view that subclasses from the generic DeleteView, remembering to import
`DeleteView `from `django.views.generic` first. Here's the code you should put in
`data_collector/view.py`:

```sh
class DeleteAlertView(DeleteView):
    template_name = 'delete_alert.html'
    model = Alert
    def get_success_url(self):
        return reverse('alerts-list')
```

Create a new `data_collector/templates/delete_alert.html` template:

```html
{% extends "base.html" %}

{% block content %}
<h1>Delete alert?</h1>
<p>Are you sure you want to delete this alert?</p>
<form action="" method="post">{% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Delete" />
    <a href="{% url 'alerts-list' %}">Cancel</a>
</form>
{% endblock %}
```

Next, import `DeleteAlertView` in `djagios/urls.py` and add this new pattern:


```sh
url(r'^alerts/(?P<pk>\d+)/delete/$', DeleteAlertView.as_view(),name='alerts-delete'),
```

Finally, let's add a link to the delete view from the alerts list page. Edit the `alerts_
list.html` template and add this line right after the Edit link:

```html
<td><a href="{% url 'alerts-delete' pk=alert.pk %}">Delete</a></td>
```

Now when you open the alerts list view, you should see a Delete link. Your screen
should look similar to the following screenshot:

![Difined Alerts new](img/Difined Alerts new2.png)

If you click on the Delete link, you should see a confirmation page. If you confirm
the deletion, you will see that your alert will be gone from the list page. These are
all the views that we will need to manage alerts. Let's move on to detecting alert
conditions and showing them in the status page.

## Showing triggered alerts on the status page



As I said before, we want our users to see any nodes that have an alert triggered
highlighted on the status page. Say that they defined an alert for when the disk usage
on any node goes beyond `0.85` and the latest data point that we have for dbmaster
disk usage has the value 0.9. When the user visits the status page, we would like the
row showing the disk usage of the dbmaster node to be highlighted in red so that the
user is immediately aware of the alert and is able to take actions to correct this.

Change `StatusView` in `data_collector/view.py` to match the following code. The
changed bits are highlighted:

```sh
class StatusView(TemplateView):
    template_name = 'status.html'
    
    def get_context_data(self, **kwargs):
        ctx = super(StatusView, self).get_context_data(**kwargs)
        
        alerts = Alert.objects.filter(is_active=True)
        
        nodes_and_data_types = DataPoint.objects.all().values('node_name', 'data_type').distinct()
        
        status_data_dict = dict()
        for node_and_data_type_pair in nodes_and_data_types:
            node_name = node_and_data_type_pair['node_name']
            data_type = node_and_data_type_pair['data_type']
            
            latest_data_point = DataPoint.objects.filter(node_name=node_name, data_type=data_type).latest('datetime')
                
            latest_data_point.has_alert = self.does_have_alert(latest_data_point, alerts)
                
            data_point_map = status_data_dict.setdefault(node_name,dict())
            data_point_map[data_type] = latest_data_point
            
        ctx['status_data_dict'] = status_data_dict
        
        return ctx
        
    def does_have_alert(self, data_point, alerts):
        for alert in alerts:
            if alert.node_name and data_point.node_name != alert.node_name:
                continue
            if alert.data_type != data_point.data_type:
                continue
            if alert.min_value is not None and data_point.data_value < alert.min_value:
                return True
            if alert.max_value is not None and data_point.data_value > alert.max_value:
                return True
        return False
```

What we have done here is to check, for each data point that we retrieve, whether it
triggers any alert. We do this by comparing the min and max values in every alert
with the data point value, but only if the data point data type and node name match
those in the alert. If the data point value is outside of the alert range, we mark the
data point as having triggered an alert.


This is another technique often used by me in a number of projects I've done. As
models are just Python objects, you can attach extra information to them on the fly.
No need to define` has_alert` on the `DataPoint` class. Just add it to the objects when
needed. Be careful though. It's not a good programming practice to do something
like this, because someone trying to understand the `DataPoint` class will have no
idea that the `has_alert` attribute event exists unless they look at the code for the
view class. As we only use this attribute in the view and template, it's fine for us.
However, if we passed the `DataPoint` objects around and more code started using
this attribute, it would be much better to define it on the class itself so that anyone
looking at the class code would know it existed.

We need to modify the `status.html` template as well to make use of the `has_alert`
attribute that we have added to the data points. Change it to match the following
code. As before, the modified portions have been highlighted:

```html
{% extends "base.html" %}

{% load humanize %}

{% block content %}
<h1>Status</h1>

<table>
    <tbody>
        <tr>
            <th>Node Name</th>
            <th>Metric</th>
            <th>Value</th>
            <th>Last Updated</th>
        </tr>
        
        {% for node_name, data_type_to_data_point_map in status_data_dict.items %}
            {% for data_type, data_point in data_type_to_data_point_map.items %}
            <tr {% if data_point.has_alert %}class="has-alert"{% endif%}>
                <td>{% if forloop.first %}{{ node_name }}{% endif %}</td>
                <td>{{ data_type }}</td>
                <td>{{ data_point.data_value }}</td>
                <td>{{ data_point.datetime|naturaltime }}</td>
            </tr>
            {% endfor %}
        {% endfor %}
    </tbody>
</table>

<style type="text/css" media="all">
    tr.has-alert td:not(:first-child) {
        color: red;
    }
</style>
{% endblock %}
```

That's it. In order to test it out, you need to create some `Alert `objects that will be
triggered by `DataPoints` in your database. For the sample data that I used, I created
one `Alert` object with the disk_usage data type and a max value of 0.5. After
creating the alert, my status screen highlighted the nodes that triggered the alert.
You screen will show something similar:

![status](img/status1.png)

To test whether our highlighting code works correctly, I added another data point
for the dbmaster disk usage metric, using the following command:

```sh
> python manage.py sample_data dbmaster disk_usage 0.2
```

After refreshing the status page, the alert condition for the dbmaster node
disappeared. You should do a similar test to see for yourself.

That's it! It's been hard work, but our monitoring tool is now starting to shape up.
We have a status page that shows the latest node statuses, highlighting any that
have alerts. The highlights disappear once the alert condition is resolved. We have
a page to manage our alerts as well. All in all, we can say that the user-facing parts
of the application are almost finished. One thing that would be pretty helpful is a
navigation bar. Add this just after the start of the `body` tag in `templates/base.html`:

```html
<ul>
    <li><a href="{% url 'status' %}">Home</a></li>
    <li><a href="{% url 'alerts-list' %}">Alerts</a></li>
</ul>
```

Refresh the status page and you should see a simple navigation menu at the top of
the page.

## Accepting data from remote systems


Now that users can see the status of their infrastructure and manage alerts, it's time
to move on to the next step: getting data from real sources instead of using Django
management commands to input sample data.

To do so, we will create an API endpoint that accepts data from remote systems. An
API endpoint is just a fancy name for a Django view that doesn't have a template to
render. The response from an API endpoint is typically either just a 200 OK status or
a JSON response. API endpoints are not meant to be used by human users. Instead,
they are meant to be used by different software systems to connect together and
share information.

The API endpoint that we need to create will be a simple view that accepts a POST
request with all the information needed to create a new DataPoint object. To make
sure that malicious users can't spam our database with random data, we will also
add a simple authentication mechanism to our API endpoint so that it will accept
data only from authorized sources.

To create an API endpoint, we'll use the `django.view.generic.View` class,
implementing only the POST handler. To parse the request data, we'll be creating a
model form on the fly. Edit `data_collector/views.py` and add the following code:

```python
from django.forms.models import modelform_factory
from django.http.response import HttpResponse
from django.http.response import HttpResponseBadRequest
from django.http.response import HttpResponseForbidden
from django.views.generic import View

class RecordDataApiView(View):
    def post(self, request, *args, **kwargs):
        # Check if the secret key matches
        if request.META.get('HTTP_AUTH_SECRET') != 'supersecretkey':
            return HttpResponseForbidden('Auth key incorrect')
        form_class = modelform_factory(DataPoint, fields=['node_name','data_type', 'data_value'])
        form = form_class(request.POST)
        if form.is_valid():
            form.save()
            return HttpResponse()
        else:
            return HttpResponseBadRequest()
```

There are a couple of new things here that we need to look at. Firstly, we have used
the `META` attribute on the request object to access a request. If you have knowledge of
how the HTTP protocol works, you should be familiar with headers. If not, a good
explanation is available at `https://www.jmarshall.com/easy/http/`. Explaining
headers in detail is beyond the scope of this book, but simply stated, headers are
extra pieces of information added to an HTTP request by the client. We'll see how to
add them in the next section when we test our API view.



Django automatically normalizes all header names and adds them to the `META`
dictionary. Here, we are using a custom header Auth-Secret to make sure that
only clients that have our secret key can use this view.

>More information about what goes into the META dictionary and how it is consturcted is available in the Django docs at https://docs.djangoproject.com/en/stable/ref/ request-response/#django.http.HttpRequest.META.

The next thing that we need to look at is the `modelform_factor`y function. It's
a neat little function provided by Django that returns a `ModelForm` subclass for
the given model. You can customize the model form to a certain degree using the
arguments to this function. Here, we limit the number of fields that can be edited.
Why use a model form in the first place?


What we wanted from the API endpoint was a method to create new `DataPoint`
models. Model forms provide exactly the functionality that we need to do this plus
they handle data validation for us. We could have created a separate model form
class in a `forms.py` file and then used that in our view as we have done before, but
there are two reasons we didn't.


Firstly, this was the only place in our code where we use a model form for the
`DataPoint` method. If we needed to use it in other places as well, defining the model
form in a single place would be the best programming practice. However, as we
don't, we can get away with defining the model form dynamically where we need it.


Secondly, we didn't need any customizations to the model form class. If we wanted
to, say, override the `save` method as we've done previously, we would have been
forced to define the class instead of using the `modelform_factory` method.

After getting the model form class, we use it like any model form class to either
create the new data point or return a response indicating that the data validation
failed. To make our new endpoint available via a URL, import the following in
djagios/urls.py:

```python
from django.views.decorators.csrf import csrf_exempt
from data_collector.views import RecordDataApiView
```

Then, add this URL pattern:


```
url(r'^record/$', csrf_exempt(RecordDataApiView.as_view()),name='record-data'),
```

The `csrf_exempt` decorator is used because, by default, Django uses CSRF
protection for POST requests. However, this is usually used on web forms, not API
endpoints. So we have to disable it, otherwise Django won't allow our POST request
to succeed. Now, let's look at how to test out our new view.

>You can get more information about CSRF protection provided by Django at `https://docs.djangoproject.com/en/stable/ref/csrf/.`

## Testing API endpoints



You can't simply test this API endpoint in your browser because it's a POST request
and there's no template that can render a form in your browser. However, a lot
of great tools are available to make manual POST requests. The one that I suggest
you use is Postman. It's a Google Chrome app, so you don't need to install any
dependencies. As long as you have Google Chrome installed on your machine, you
can get Postman from `https://www.getpostman.com/`. After you have installed
it, start it up and you should see a screen similar to the following one. Don't
worry if your Postman screen isn't exactly the same. It can be that the version you
downloaded is a newer one. The main parts of Postman should be the same.

![Testing API endpoints_1](img/Testing API endpoints_1.png)

Using Postman is simple. I will walk you through the entire process step by step,
including an image for each step to make it clear what I mean. At the end of this
process, we should be able to use our API endpoint to generate a new data point.


As a side note, if you are using a Linux- or Unix-based OS, such as Ubuntu or Mac
OS X, and are more comfortable using the command line, you can use the curl
utility to make the POST request. It is usually faster for simpler requests. To make
the same request using `curl` as the one I demonstrate here in Postman, type this on
your command prompt:

```sh
> curl http://127.0.0.1:8000/record/ -H 'Auth-Secret: supersecretkey' -d
node_name=web01 -d data_type=disk_usage -d data_value=0.2
```

To make this request using Postman, perform the following steps:

1. Select the request type. We want to make a POST request, so select POST from the drop-down menu:

![Testing API endpoints_2](img/Testing API endpoints_2.png)

2.Enter the URL you want to make the request to. In our case, it is
http://127.0.0.1:8000/record/:

![Testing API endpoints_3](img/Testing API endpoints_3.png)

3.Add our custom authentication header. Open up the Headers tab and add the Auth-Secret header with a value of supersecretkey:

![Testing API endpoints_4](img/Testing API endpoints_4.png)

4.Finally, add our POST parameters to the Body section. The sample data that I used was as follows:
- node_name: web01
- data_type: disk_usage
-  data_value: 0.72

![Testing API endpoints_5](img/Testing API endpoints_5.png)

That's it. Our request is now set up. Click on the Send button next to the URL textbox
and you should see an empty response below the body parameters. To confirm that
the request worked properly, look at the status code of the response.
It should be 200 OK:

![Testing API endpoints_6](img/Testing API endpoints_6.png)

Open up the status page for our application at `http://127.0.0.1:8000/` and you
should see the latest data point value displayed there. That's it, we're done!

> As explained at the start of this section, you can also use
command-line tools such as `curl` to upload data to the
API. Using such tools, you can come up with shell scripts
that automatically update the web app with real data from
your computer systems. This is how Nagios and a lot of data
monitoring tools operate as well. The servers have API endpoints
that listen for data, and then simple scripts gather and upload
data to the servers from the client nodes.

## Summary



This was a pretty demanding chapter, and you learned a lot of new information.



For starters, we looked at Django management commands. They are an important
feature of Django. All Django commands that you run, such as `python manage.py`
startapp, are management commands as well, so you should already know how
powerful they can be. In bigger projects, you almost always have a few management
commands to automate your tasks for you.



We also looked at how Django creates a string representation of our models using
the `__str__` method on our model class. It's not just used when printing to the
console either. Any time you try to use your model objects as strings, even in your
templates, Django uses this representation, so it's important to have a good format
that can immediately give you all the important information about an object.



This chapter also introduced advance query methods, specifically the `distinct` and
`values` methods that allow you to issue more complicated SQL queries to get data in
exactly the format you want from the database. However, this is just the beginning.
In later chapters, we might need to use more complicated query methods. You might
want to look at the Django documentation for `queryset` methods at `https://docs.djangoproject.com/en/stable/ref/models/querysets/` to learn about more
methods of querying the databases that Django provides.



In addition to getting data from the database in a format that we wanted, we also
looked at preparing a somewhat complicated data structure with the data to pass all
the required information to templates, and then saw how to use that data structure in
our templates.



Often, you have complicated data validation rules that you need to ensure are
passing before you can save data to a database. We looked at how to achieve
this in this chapter by overriding the `save` method of our model classes.



Finally, you learned how to create simple API endpoints and how to test them
using `curl` or Postman. Overall, this was a big chapter that introduced a lot of
new concepts that will be used in later chapters.